In [2]:
# Load the data into a dataframe
data_flights <- read.csv('../Data/CustomerFlightActivity.csv')
dim(data_flights)

# Print the total amount of 'NA's in the data
sum(is.na(data_flights))

[1] 405624     10

[1] 0

In [3]:
head(data_flights)

,Loyalty.Number,Year,Month,Flights.Booked,Flights.with.Companions,Total.Flights,Distance,Points.Accumulated,Points.Redeemed,Dollar.Cost.Points.Redeemed
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>
1,100018,2017,1,3,0,3,1521,152,0,0
2,100102,2017,1,10,4,14,2030,203,0,0
3,100140,2017,1,6,0,6,1200,120,0,0
4,100214,2017,1,0,0,0,0,0,0,0
5,100272,2017,1,0,0,0,0,0,0,0
6,100301,2017,1,0,0,0,0,0,0,0


In [ ]:
data_loyalty <- read.csv('../Data/CustomerLoyaltyHistory.csv')
dim(data_loyalty)
sum(is.na(data_loyalty))